In [1]:
import plotly.express as px
import plotly.graph_objects as go
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import datetime as dt
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import seaborn as sns
import sqlite3

# warnings
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

# Hide warning temporarily
pd.options.mode.chained_assignment = None

In [2]:
df = pd.read_csv("data/flights_sample_2m.csv")

In [3]:
dow_category = CategoricalDtype(categories=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday",
                                            "Saturday", "Sunday"], ordered=True)

df["FL_DATE"] = pd.to_datetime(df["FL_DATE"])
df["YEAR"] = df["FL_DATE"].dt.year
df["MONTH"] = df["FL_DATE"].dt.month
df["QUARTER"] = ((df["MONTH"] -1)//3)+1
df["DAY_OF_WEEK"] = df["FL_DATE"].dt.dayofweek.map({0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday',
                                                     4: 'Friday', 5: 'Saturday', 6: 'Sunday'}).astype(dow_category)

df["DISTANCE_CAT"] = pd.cut(
    df["DISTANCE"], bins = [-1, 500, 1500, float("inf")],
    labels = ["Short-haul", "Medium-haul", "Long-haul"]
)

In [4]:
# Tạo file database mới (nếu chưa có)
conn = sqlite3.connect("flights.db")  # file sẽ tự động tạo

# Ghi DataFrame vào SQLite
df.to_sql("flights", conn, if_exists="replace", index=False)

2000000

In [5]:
query_1 = """
SELECT R1.TOTAL_FLIGHTS, R2.N_DELAYED_FLIGHTS, R3.N_CANCELLED_FLIGHTS, R4.N_DIVERTED_FLIGHTS, R5.N_ONTIME_FLIGHTS
FROM (SELECT printf("%,d", COUNT(*)) AS TOTAL_FLIGHTS FROM flights) R1
JOIN (SELECT printf("%,d", COUNT(*)) AS N_DELAYED_FLIGHTS FROM flights WHERE ARR_DELAY > 15) R2
JOIN (SELECT printf("%,d", COUNT(*)) AS N_CANCELLED_FLIGHTS FROM flights WHERE CANCELLED == 1) R3
JOIN (SELECT printf("%,d", COUNT(*)) AS N_DIVERTED_FLIGHTS FROM flights WHERE DIVERTED == 1) R4
JOIN (SELECT printf("%,d", COUNT(*)) AS N_ONTIME_FLIGHTS FROM flights WHERE ARR_DELAY <=15) R5
"""
result_1 = pd.read_sql(query_1, conn)
result_1

,TOTAL_FLIGHTS,N_DELAYED_FLIGHTS,N_CANCELLED_FLIGHTS,N_DIVERTED_FLIGHTS,N_ONTIME_FLIGHTS
0,"2,000,000","343,875","52,522","4,709","1,598,894"


In [6]:
query_2 = """
SELECT DISTINCT AIRLINE
FROM flights
ORDER BY AIRLINE;
"""
result_2 = pd.read_sql(query_2, conn)
result_2

,AIRLINE
0,Alaska Airlines Inc.
1,Allegiant Air
2,American Airlines Inc.
3,Delta Air Lines Inc.
4,Endeavor Air Inc.
5,Envoy Air
6,ExpressJet Airlines LLC d/b/a aha!
7,Frontier Airlines Inc.
8,Hawaiian Airlines Inc.
9,Horizon Air


In [10]:
query_3 = """
SELECT YEAR, printf("%,d", COUNT(*)) AS TOTAL_FLIGHTS
FROM flights
GROUP BY YEAR;
"""
result_3 = pd.read_sql(query_3, conn)
result_3

,YEAR,TOTAL_FLIGHTS
0,2019,"505,345"
1,2020,"318,768"
2,2021,"407,824"
3,2022,"457,760"
4,2023,"310,303"


In [11]:
query_4 = """
SELECT YEAR, MONTH, printf("%,d", COUNT(*)) AS TOTAL_FLIGHTS
FROM flights
GROUP BY YEAR, MONTH;
"""
result_4 = pd.read_sql(query_4, conn)
result_4

,YEAR,MONTH,TOTAL_FLIGHTS
0,2019,1,"39,568"
1,2019,2,"35,912"
2,2019,3,"43,208"
3,2019,4,"41,136"
4,2019,5,"43,293"
5,2019,6,"43,584"
6,2019,7,"44,850"
7,2019,8,"45,015"
8,2019,9,"41,281"
9,2019,10,"43,438"


In [12]:
query_5 = """
SELECT YEAR, printf("%,d", COUNT(*)) AS TOTAL_FLIGHTS
FROM flights
WHERE AIRLINE == 'Envoy Air'
GROUP BY YEAR;
"""
result_5 = pd.read_sql(query_5, conn)
result_5

,YEAR,TOTAL_FLIGHTS
0,2019,"22,111"
1,2020,"14,435"
2,2021,"17,665"
3,2022,"16,823"
4,2023,"10,111"


In [13]:
query_6 = """
SELECT YEAR, MONTH, printf("%,d", COUNT(*)) AS TOTAL_FLIGHTS
FROM flights
WHERE AIRLINE == 'Envoy Air'
GROUP BY YEAR, MONTH;
"""
result_6 = pd.read_sql(query_6, conn)
result_6

,YEAR,MONTH,TOTAL_FLIGHTS
0,2019,1,"1,751"
1,2019,2,"1,531"
2,2019,3,"1,785"
3,2019,4,"1,761"
4,2019,5,"1,933"
5,2019,6,"1,923"
6,2019,7,"1,999"
7,2019,8,"2,106"
8,2019,9,"1,877"
9,2019,10,"1,875"
